# Pruebas Replacement Policies
## Función
Con esta función se van a probar los replacement policies. Todos los subdominios tienen una caché de 1000 bytes.

In [ ]:
import requests
import time

def send_request(url):
    try:
        response = requests.get("http://" + url)
        if response.status_code == 200:
            print("Request successful!")
            print("Response:", response.text)
        else:
            print(f"Request failed with status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

## Least Recently Used

In [11]:
# LRU
send_request("lru.mooo.com/getPokemon/001")
time.sleep(2)
send_request("lru.mooo.com/getPokemon/002")
time.sleep(2)
send_request("lru.mooo.com/getPokemon/001")

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type1":"Grass","Type2":"Poison","Weightkg":"6.9","Weightlbs":"15.2"}

Request successful!
Response: {"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"}

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type

Para este punto, la caché se debería ver así
```json
{
  "lru.mooo.com": {
    "size": 980,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 00:03:07 2025",
        "most_recent_use": "Sat Jun  7 00:03:11 2025",
        "times_used": 1,
        "time_to_live": "Sat Jun  7 00:18:07 2025",
        "filename": "b8dfb770647a038a6d36a56f3a8971e1cd8930125effbc72e555a0c255ca9b50.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 00:03:09 2025",
        "most_recent_use": "Sat Jun  7 00:03:09 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 00:18:09 2025",
        "filename": "587111aa5b86d7ee32853df7e4d9b4ff504e06325801d0b19558e2fc4c15e7a5.ksh",
        "size": 489
      }
    }
  }
}
```

Como podemos ver, el que se usó menos recientemente es el registro 002, con 2 segundos de diferencia

In [12]:
send_request("lru.mooo.com/getPokemon/003")

Request successful!
Response: {"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"}



El resultado es:
```
Key to delete: GET-host.docker.internal:5000/getPokemon/002
Added URI to cache for host: lru.mooo.com
Cache Object: 
```
```json
{
  "lru.mooo.com": {
    "size": 985,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 00:03:07 2025",
        "most_recent_use": "Sat Jun  7 00:03:11 2025",
        "times_used": 1,
        "time_to_live": "Sat Jun  7 00:18:07 2025",
        "filename": "b8dfb770647a038a6d36a56f3a8971e1cd8930125effbc72e555a0c255ca9b50.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 00:04:30 2025",
        "most_recent_use": "Sat Jun  7 00:04:30 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 00:19:30 2025",
        "filename": "6a874af9edc731480933e7a7bc1734657f434aafb4ce3de35d28535a2182d33b.ksh",
        "size": 494
      }
    }
  }
}
```

Como podemos, ver se sacó el 002

El siguiente request retorna todos los Pokémones, cuya respuesta es tan grande que saca todo de la cache para intentar meterlo, pero al final igualmente no lo puede meter, entonces termina vaciándose la caché.

In [18]:
send_request("lru.mooo.com/getAllPokemon") 

Request successful!
Response: [{"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"},{"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"},{"Attack":"52","CaptureRate":"45","Category":"Lizard Pok\u00e9mon","Defense":"43","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"39","Heightf":"2'00\"","Heightm":"0.6","Id":"004","Name":"Charmander","SpAttack":"60","SpDefense":"50","Speed":"65","Total":"309","Type1":"Fire","Type2":"None","Weightkg":"8.5","Weightlbs

## Least Frequently Used

In [19]:
# LFU
send_request("lfu.mooo.com/getPokemon/001")
time.sleep(2)
send_request("lfu.mooo.com/getPokemon/002")
time.sleep(2)
send_request("lfu.mooo.com/getPokemon/001")

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type1":"Grass","Type2":"Poison","Weightkg":"6.9","Weightlbs":"15.2"}

Request successful!
Response: {"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"}

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type

La caché se ve así
```json
{
  "lfu.mooo.com": {
    "size": 980,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 00:40:43 2025",
        "most_recent_use": "Sat Jun  7 00:40:47 2025",
        "times_used": 1,
        "time_to_live": "Sat Jun  7 00:55:43 2025",
        "filename": "720c7e0e2b0b96e2bb5ec91a211e168e6e8c487d733a17d47252c3297e94f54f.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 00:40:45 2025",
        "most_recent_use": "Sat Jun  7 00:40:45 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 00:55:45 2025",
        "filename": "f486efeb673715045963a411afd2c1753ac47efa491e50bc60cc54651d794a74.ksh",
        "size": 489
      }
    }
  }
}
```

Como podemos ver, el registro menos usado es el 002

In [20]:
send_request("lfu.mooo.com/getPokemon/003")

Request successful!
Response: {"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"}



Los resultados son:
```
Least used count: 1 with times_used: 0
Key to delete: GET-host.docker.internal:5000/getPokemon/002
Added URI to cache for host: lfu.mooo.com
Cache Object: 
```
```json
{
  "lfu.mooo.com": {
    "size": 985,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 00:40:43 2025",
        "most_recent_use": "Sat Jun  7 00:40:47 2025",
        "times_used": 1,
        "time_to_live": "Sat Jun  7 00:55:43 2025",
        "filename": "720c7e0e2b0b96e2bb5ec91a211e168e6e8c487d733a17d47252c3297e94f54f.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 00:43:06 2025",
        "most_recent_use": "Sat Jun  7 00:43:06 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 00:58:06 2025",
        "filename": "e2fef87e65aaac4f2a72819742af69fe7c703f32099429112de111c3fe616ed1.ksh",
        "size": 494
      }
    }
  }
}
```
Se eliminó el registro 002 correctamente

In [21]:
send_request("lfu.mooo.com/getAllPokemon")

Request successful!
Response: [{"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"},{"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"},{"Attack":"52","CaptureRate":"45","Category":"Lizard Pok\u00e9mon","Defense":"43","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"39","Heightf":"2'00\"","Heightm":"0.6","Id":"004","Name":"Charmander","SpAttack":"60","SpDefense":"50","Speed":"65","Total":"309","Type1":"Fire","Type2":"None","Weightkg":"8.5","Weightlbs

## First In, First Out

In [22]:
# FIFO
send_request("fifo.mooo.com/getPokemon/001")
time.sleep(2)
send_request("fifo.mooo.com/getPokemon/002")
time.sleep(2)
send_request("fifo.mooo.com/getPokemon/001")

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type1":"Grass","Type2":"Poison","Weightkg":"6.9","Weightlbs":"15.2"}

Request successful!
Response: {"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"}

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type

Así se va la caché
```json
{
  "fifo.mooo.com": {
    "size": 980,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 00:45:12 2025",
        "most_recent_use": "Sat Jun  7 00:45:17 2025",
        "times_used": 1,
        "time_to_live": "Sat Jun  7 01:00:12 2025",
        "filename": "9fca83b35283f3833a57044069784dbdfe2e73e5fb7b2ee6327f96dff47afc0a.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 00:45:14 2025",
        "most_recent_use": "Sat Jun  7 00:45:14 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 01:00:14 2025",
        "filename": "ef5e17541ba65c79b9f3e1341004f803d81e45d51c3c0d7d802ea26709ee3079.ksh",
        "size": 489
      }
    }
  }
}
```
El registro más antiguo es el 001.

In [23]:
send_request("fifo.mooo.com/getPokemon/003")

Request successful!
Response: {"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"}



Los resultados son
```
Implementing FIFO replacement policy.
Key to delete: GET-host.docker.internal:5000/getPokemon/001
Added URI to cache for host: fifo.mooo.com
Cache Object:
```
```json
{
  "fifo.mooo.com": {
    "size": 983,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 00:45:14 2025",
        "most_recent_use": "Sat Jun  7 00:45:14 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 01:00:14 2025",
        "filename": "ef5e17541ba65c79b9f3e1341004f803d81e45d51c3c0d7d802ea26709ee3079.ksh",
        "size": 489
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 00:49:45 2025",
        "most_recent_use": "Sat Jun  7 00:49:45 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 01:04:45 2025",
        "filename": "1e6732a8cd8734318bf80f6763d116ffff5451b177a66d1dacd2f5b916a96fb5.ksh",
        "size": 494
      }
    }
  }
}
```
Como podemos ver, se eliminó correctamente el registro 1.

In [24]:
send_request("fifo.mooo.com/getAllPokemon")

Request successful!
Response: [{"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"},{"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"},{"Attack":"52","CaptureRate":"45","Category":"Lizard Pok\u00e9mon","Defense":"43","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"39","Heightf":"2'00\"","Heightm":"0.6","Id":"004","Name":"Charmander","SpAttack":"60","SpDefense":"50","Speed":"65","Total":"309","Type1":"Fire","Type2":"None","Weightkg":"8.5","Weightlbs

## Most Recently Used

In [25]:
# MRU
send_request("mru.mooo.com/getPokemon/001")
time.sleep(2)
send_request("mru.mooo.com/getPokemon/002")
time.sleep(2)
send_request("mru.mooo.com/getPokemon/001")

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type1":"Grass","Type2":"Poison","Weightkg":"6.9","Weightlbs":"15.2"}

Request successful!
Response: {"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"}

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type

La caché se ve así
```json
{
  "mru.mooo.com": {
    "size": 980,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 00:58:49 2025",
        "most_recent_use": "Sat Jun  7 00:58:53 2025",
        "times_used": 1,
        "time_to_live": "Sat Jun  7 01:13:49 2025",
        "filename": "e8067a4500a37fce26682bc722aaff1f3fa0404b6a05537bdfdb72387d955ee6.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 00:58:51 2025",
        "most_recent_use": "Sat Jun  7 00:58:51 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 01:13:51 2025",
        "filename": "4c6fd7eb1980a0d4c9a9ff1136ff0a71eca9c6322c979d53727dd2ebf24ecbc8.ksh",
        "size": 489
      }
    }
  }
}
```
Como podemos ver, el registro usado más recientemente es el 1.

In [26]:
send_request("mru.mooo.com/getPokemon/003")

Request successful!
Response: {"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"}



Los resultados son
```
Key to delete: GET-host.docker.internal:5000/getPokemon/001
Added URI to cache for host: mru.mooo.com
Cache Object: 
```
```json
{
  "mru.mooo.com": {
    "size": 983,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 00:58:51 2025",
        "most_recent_use": "Sat Jun  7 00:58:51 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 01:13:51 2025",
        "filename": "4c6fd7eb1980a0d4c9a9ff1136ff0a71eca9c6322c979d53727dd2ebf24ecbc8.ksh",
        "size": 489
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 01:01:38 2025",
        "most_recent_use": "Sat Jun  7 01:01:38 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 01:16:38 2025",
        "filename": "0d2e5b621add3db1ce6529e84192389a72e5423044f194d5b312e58bdeed0769.ksh",
        "size": 494
      }
    }
  }
}
```
Se eliminó correctamente el registro 001.

In [27]:
send_request("fifo.mooo.com/getAllPokemon")

Request successful!
Response: [{"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"},{"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"},{"Attack":"52","CaptureRate":"45","Category":"Lizard Pok\u00e9mon","Defense":"43","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"39","Heightf":"2'00\"","Heightm":"0.6","Id":"004","Name":"Charmander","SpAttack":"60","SpDefense":"50","Speed":"65","Total":"309","Type1":"Fire","Type2":"None","Weightkg":"8.5","Weightlbs

## Random

In [31]:
# Random
send_request("random.mooo.com/getPokemon/001")
time.sleep(2)
send_request("random.mooo.com/getPokemon/002")
time.sleep(2)
send_request("random.mooo.com/getPokemon/003")
time.sleep(2)
send_request("random.mooo.com/getPokemon/004")
time.sleep(2)
send_request("random.mooo.com/getPokemon/005")
time.sleep(2)
send_request("random.mooo.com/getPokemon/006")
time.sleep(2)
send_request("random.mooo.com/getPokemon/007")
time.sleep(2)
send_request("random.mooo.com/getPokemon/008")

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type1":"Grass","Type2":"Poison","Weightkg":"6.9","Weightlbs":"15.2"}

Request successful!
Response: {"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"}

Request successful!
Response: {"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Ty

La caché de esta prueba tiene espacio para 8 registros. La caché se ve así.
```json
{
  "random.mooo.com": {
    "size": 3947,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 02:36:58 2025",
        "most_recent_use": "Sat Jun  7 02:36:58 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:51:58 2025",
        "filename": "e38b2045593609994d0f6fcfffd3b8480af814e496dc8737bb77774fc47f5e64.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 02:37:00 2025",
        "most_recent_use": "Sat Jun  7 02:37:00 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:00 2025",
        "filename": "892ec0ab9e60e9a06d04f81a990eb465e1931e5a5718272d3a2ebc4c2ac56e13.ksh",
        "size": 489
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 02:37:03 2025",
        "most_recent_use": "Sat Jun  7 02:37:03 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:03 2025",
        "filename": "9a7ef5835b4df9ca771888d3498f44d84ad2e1f07c11275804a01de6b7e1ae76.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/004": {
        "received": "Sat Jun  7 02:37:05 2025",
        "most_recent_use": "Sat Jun  7 02:37:05 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:05 2025",
        "filename": "6f65a739d5827e44b6efe9b9f09bcf2bf99ec876e1de996e4503838b33605818.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/005": {
        "received": "Sat Jun  7 02:37:07 2025",
        "most_recent_use": "Sat Jun  7 02:37:07 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:07 2025",
        "filename": "3d13f2ce1cb14bd697358e5d62e3dabb37d3563d4e01f80eee4ea4384b8585a7.ksh",
        "size": 492
      },
      "GET-host.docker.internal:5000/getPokemon/006": {
        "received": "Sat Jun  7 02:37:09 2025",
        "most_recent_use": "Sat Jun  7 02:37:09 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:09 2025",
        "filename": "f6659a46a5afb7a207ff7c397c314969de93dc7a068b1b3a321598bd38996c74.ksh",
        "size": 498
      },
      "GET-host.docker.internal:5000/getPokemon/007": {
        "received": "Sat Jun  7 02:37:11 2025",
        "most_recent_use": "Sat Jun  7 02:37:11 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:11 2025",
        "filename": "444562ee207244cfc4048fbd7cd2eb790fd99d05c993fb4eb0531051d659ffb4.ksh",
        "size": 496
      },
      "GET-host.docker.internal:5000/getPokemon/008": {
        "received": "Sat Jun  7 02:37:13 2025",
        "most_recent_use": "Sat Jun  7 02:37:13 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:13 2025",
        "filename": "e7d08104f5381cb92f6e33b26cc6cf8a3f3b0f01af770b012eee802a963752b4.ksh",
        "size": 493
      }
    }
  }
}
```

In [32]:
send_request("random.mooo.com/getPokemon/009")

Request successful!
Response: {"Attack":"83","CaptureRate":"45","Category":"Shellfish Pok\u00e9mon","Defense":"100","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"79","Heightf":"5'03\"","Heightm":"1.6","Id":"009","Name":"Blastoise","SpAttack":"85","SpDefense":"105","Speed":"78","Total":"530","Type1":"Water","Type2":"None","Weightkg":"85.5","Weightlbs":"188.5"}



Los resultados son los siguientes:
```
Key to delete: GET-host.docker.internal:5000/getPokemon/007
Added URI to cache for host: random.mooo.com
Cache Object: 
```
```json
{
  "random.mooo.com": {
    "size": 3952,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 02:36:58 2025",
        "most_recent_use": "Sat Jun  7 02:36:58 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:51:58 2025",
        "filename": "e38b2045593609994d0f6fcfffd3b8480af814e496dc8737bb77774fc47f5e64.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 02:37:00 2025",
        "most_recent_use": "Sat Jun  7 02:37:00 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:00 2025",
        "filename": "892ec0ab9e60e9a06d04f81a990eb465e1931e5a5718272d3a2ebc4c2ac56e13.ksh",
        "size": 489
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 02:37:03 2025",
        "most_recent_use": "Sat Jun  7 02:37:03 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:03 2025",
        "filename": "9a7ef5835b4df9ca771888d3498f44d84ad2e1f07c11275804a01de6b7e1ae76.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/004": {
        "received": "Sat Jun  7 02:37:05 2025",
        "most_recent_use": "Sat Jun  7 02:37:05 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:05 2025",
        "filename": "6f65a739d5827e44b6efe9b9f09bcf2bf99ec876e1de996e4503838b33605818.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/005": {
        "received": "Sat Jun  7 02:37:07 2025",
        "most_recent_use": "Sat Jun  7 02:37:07 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:07 2025",
        "filename": "3d13f2ce1cb14bd697358e5d62e3dabb37d3563d4e01f80eee4ea4384b8585a7.ksh",
        "size": 492
      },
      "GET-host.docker.internal:5000/getPokemon/006": {
        "received": "Sat Jun  7 02:37:09 2025",
        "most_recent_use": "Sat Jun  7 02:37:09 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:09 2025",
        "filename": "f6659a46a5afb7a207ff7c397c314969de93dc7a068b1b3a321598bd38996c74.ksh",
        "size": 498
      },
      "GET-host.docker.internal:5000/getPokemon/008": {
        "received": "Sat Jun  7 02:37:13 2025",
        "most_recent_use": "Sat Jun  7 02:37:13 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:52:13 2025",
        "filename": "e7d08104f5381cb92f6e33b26cc6cf8a3f3b0f01af770b012eee802a963752b4.ksh",
        "size": 493
      },
      "GET-host.docker.internal:5000/getPokemon/009": {
        "received": "Sat Jun  7 02:38:39 2025",
        "most_recent_use": "Sat Jun  7 02:38:39 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:53:39 2025",
        "filename": "ee436e1cd53dd3b87e9d181305dfd1f645aba03c03599eb0b7b4a8afdf5d330e.ksh",
        "size": 501
      }
    }
  }
}
```
Vemos que se escogió aleatoriamente el registro 007 para eliminar. Ahora probémoslo otra vez para verificar que da uno diferente.

In [34]:
send_request("random.mooo.com/getAllPokemon")

Request successful!
Response: [{"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"},{"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Type1":"Grass","Type2":"Poison","Weightkg":"100","Weightlbs":"220.5"},{"Attack":"52","CaptureRate":"45","Category":"Lizard Pok\u00e9mon","Defense":"43","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"39","Heightf":"2'00\"","Heightm":"0.6","Id":"004","Name":"Charmander","SpAttack":"60","SpDefense":"50","Speed":"65","Total":"309","Type1":"Fire","Type2":"None","Weightkg":"8.5","Weightlbs

In [35]:
send_request("random.mooo.com/getPokemon/001")
time.sleep(2)
send_request("random.mooo.com/getPokemon/002")
time.sleep(2)
send_request("random.mooo.com/getPokemon/003")
time.sleep(2)
send_request("random.mooo.com/getPokemon/004")
time.sleep(2)
send_request("random.mooo.com/getPokemon/005")
time.sleep(2)
send_request("random.mooo.com/getPokemon/006")
time.sleep(2)
send_request("random.mooo.com/getPokemon/007")
time.sleep(2)
send_request("random.mooo.com/getPokemon/008")

Request successful!
Response: {"Attack":"49","CaptureRate":"45","Category":"Seed Pokemon","Defense":"49","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"45","Heightf":"2'04\\\"","Heightm":"0.7","Id":"001","Name":"Bulbasaur","SpAttack":"65","SpDefense":"65","Speed":"45","Total":"318","Type1":"Grass","Type2":"Poison","Weightkg":"6.9","Weightlbs":"15.2"}

Request successful!
Response: {"Attack":"62","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"63","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"60","Heightf":"3'03\"","Heightm":"1","Id":"002","Name":"Ivysaur","SpAttack":"80","SpDefense":"80","Speed":"60","Total":"405","Type1":"Grass","Type2":"Poison","Weightkg":"13","Weightlbs":"28.7"}

Request successful!
Response: {"Attack":"82","CaptureRate":"45","Category":"Seed Pok\u00e9mon","Defense":"83","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"80","Heightf":"6'07\"","Heightm":"2","Id":"003","Name":"Venusaur","SpAttack":"100","SpDefense":"100","Speed":"80","Total":"525","Ty

La caché se ve así
```json
{
  "random.mooo.com": {
    "size": 3947,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 02:40:27 2025",
        "most_recent_use": "Sat Jun  7 02:40:27 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:27 2025",
        "filename": "2c254fa119b9df65f5d2cabae6833198e42dc9057dbe7718b2aedc855f132f25.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 02:40:29 2025",
        "most_recent_use": "Sat Jun  7 02:40:29 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:29 2025",
        "filename": "7dfa056b49d377249ee967c90f66e43e165a2e2db015a4c345d455d5af7e78e1.ksh",
        "size": 489
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 02:40:31 2025",
        "most_recent_use": "Sat Jun  7 02:40:31 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:31 2025",
        "filename": "75798a5163f506e3991bec7e8212d1323a33a037693c251a3f2ba14dc2502756.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/004": {
        "received": "Sat Jun  7 02:40:33 2025",
        "most_recent_use": "Sat Jun  7 02:40:33 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:33 2025",
        "filename": "d37b7e1ce3e72777e009c0cb77d1093e87cf37cca80e644d366d7e03295bb2f2.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/005": {
        "received": "Sat Jun  7 02:40:35 2025",
        "most_recent_use": "Sat Jun  7 02:40:35 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:35 2025",
        "filename": "81f9d6dfbc4359a3791befa7b603ecdf3b5c48cf87561128528c2246a4c088c5.ksh",
        "size": 492
      },
      "GET-host.docker.internal:5000/getPokemon/006": {
        "received": "Sat Jun  7 02:40:37 2025",
        "most_recent_use": "Sat Jun  7 02:40:37 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:37 2025",
        "filename": "e91984ed65add8c1b1009d5f82ccdfca187202c8000a556161fe1efbe1e9220a.ksh",
        "size": 498
      },
      "GET-host.docker.internal:5000/getPokemon/007": {
        "received": "Sat Jun  7 02:40:39 2025",
        "most_recent_use": "Sat Jun  7 02:40:39 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:39 2025",
        "filename": "287dc57312e853a730c91410476c4514da61bb50bb90b0b2591f659af96a9c11.ksh",
        "size": 496
      },
      "GET-host.docker.internal:5000/getPokemon/008": {
        "received": "Sat Jun  7 02:40:41 2025",
        "most_recent_use": "Sat Jun  7 02:40:41 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:41 2025",
        "filename": "be028748cccab49908ad2f6ccd56e802f4fc84ee4be63160ff3ae51cc8ecc885.ksh",
        "size": 493
      }
    }
  }
}
```

In [36]:
send_request("random.mooo.com/getPokemon/009")

Request successful!
Response: {"Attack":"83","CaptureRate":"45","Category":"Shellfish Pok\u00e9mon","Defense":"100","EggSteps":"5120","ExpGroup":"Medium Slow","HP":"79","Heightf":"5'03\"","Heightm":"1.6","Id":"009","Name":"Blastoise","SpAttack":"85","SpDefense":"105","Speed":"78","Total":"530","Type1":"Water","Type2":"None","Weightkg":"85.5","Weightlbs":"188.5"}



Los resultados son los siguientes:
```
Implementing Random replacement policy.
Key to delete: GET-host.docker.internal:5000/getPokemon/004
Added URI to cache for host: random.mooo.com
Cache Object:
```
```json
{
  "random.mooo.com": {
    "size": 3954,
    "requests": {
      "GET-host.docker.internal:5000/getPokemon/001": {
        "received": "Sat Jun  7 02:40:27 2025",
        "most_recent_use": "Sat Jun  7 02:40:27 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:27 2025",
        "filename": "2c254fa119b9df65f5d2cabae6833198e42dc9057dbe7718b2aedc855f132f25.ksh",
        "size": 491
      },
      "GET-host.docker.internal:5000/getPokemon/002": {
        "received": "Sat Jun  7 02:40:29 2025",
        "most_recent_use": "Sat Jun  7 02:40:29 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:29 2025",
        "filename": "7dfa056b49d377249ee967c90f66e43e165a2e2db015a4c345d455d5af7e78e1.ksh",
        "size": 489
      },
      "GET-host.docker.internal:5000/getPokemon/003": {
        "received": "Sat Jun  7 02:40:31 2025",
        "most_recent_use": "Sat Jun  7 02:40:31 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:31 2025",
        "filename": "75798a5163f506e3991bec7e8212d1323a33a037693c251a3f2ba14dc2502756.ksh",
        "size": 494
      },
      "GET-host.docker.internal:5000/getPokemon/008": {
        "received": "Sat Jun  7 02:40:41 2025",
        "most_recent_use": "Sat Jun  7 02:40:41 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:41 2025",
        "filename": "be028748cccab49908ad2f6ccd56e802f4fc84ee4be63160ff3ae51cc8ecc885.ksh",
        "size": 493
      },
      "GET-host.docker.internal:5000/getPokemon/005": {
        "received": "Sat Jun  7 02:40:35 2025",
        "most_recent_use": "Sat Jun  7 02:40:35 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:35 2025",
        "filename": "81f9d6dfbc4359a3791befa7b603ecdf3b5c48cf87561128528c2246a4c088c5.ksh",
        "size": 492
      },
      "GET-host.docker.internal:5000/getPokemon/006": {
        "received": "Sat Jun  7 02:40:37 2025",
        "most_recent_use": "Sat Jun  7 02:40:37 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:37 2025",
        "filename": "e91984ed65add8c1b1009d5f82ccdfca187202c8000a556161fe1efbe1e9220a.ksh",
        "size": 498
      },
      "GET-host.docker.internal:5000/getPokemon/007": {
        "received": "Sat Jun  7 02:40:39 2025",
        "most_recent_use": "Sat Jun  7 02:40:39 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:55:39 2025",
        "filename": "287dc57312e853a730c91410476c4514da61bb50bb90b0b2591f659af96a9c11.ksh",
        "size": 496
      },
      "GET-host.docker.internal:5000/getPokemon/009": {
        "received": "Sat Jun  7 02:41:30 2025",
        "most_recent_use": "Sat Jun  7 02:41:30 2025",
        "times_used": 0,
        "time_to_live": "Sat Jun  7 02:56:30 2025",
        "filename": "343fa4e854c042b767ee1212a2b1f79227ffa1fbb7fa07b71f90e3b22d32a79b.ksh",
        "size": 501
      }
    }
  }
}
```
Como podemos ver, esta vez se escogió aleatoriamente el registro 004 para eliminar.

Por lo tanto, todas las replacement policies están funcionando correctamente.